**Purpose of Notebook:** **Importing excel Tables from** '<u><mark>Data Taables .xlxs</mark></u>' **file using Pandas**

**kernal Python 3**

**Created by:** <span style="background-color: rgba(127, 127, 127, 0.1);">Shabba</span>

**Date:** <span style="background-color: rgba(127, 127, 127, 0.1);">23-05-202</span><span style="background-color: rgba(127, 127, 127, 0.1);">3</span>